In [22]:
import pandas as pd
import pyarrow
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()

try:
    # Conexión a SalesSystem (Facturación)
    salessystem_url = f"{os.getenv('DB_SALESSYSTEM_DIALECT')}://{os.getenv('DB_SALESSYSTEM_USER')}:{os.getenv('DB_SALESSYSTEM_PASSWORD')}@{os.getenv('DB_SALESSYSTEM_HOST')}:{os.getenv('DB_SALESSYSTEM_PORT')}/{os.getenv('DB_SALESSYSTEM_NAME')}"
    salessystem = create_engine(salessystem_url)
    
    # Conexión a Warehouse (Contabilidad)
    warehouse_url = f"{os.getenv('DB_WAREHOUSE_DIALECT')}://{os.getenv('DB_WAREHOUSE_USER')}:{os.getenv('DB_WAREHOUSE_PASSWORD')}@{os.getenv('DB_WAREHOUSE_HOST')}:{os.getenv('DB_WAREHOUSE_PORT')}/{os.getenv('DB_WAREHOUSEE_NAME')}"
    warehouse = create_engine(warehouse_url)
    
    # Validar las conexiones
    with salessystem.connect() as conn_sales:
        pass
    with warehouse.connect() as conn_warehouse:
        pass
        
except Exception as e:
    print(f"Error al conectar a las bases de datos: {str(e)}")
    raise

In [23]:
# VISTA FACTURAS_NOANULADAS FILTRADAS POR PERIODO '202406'
query = ("SELECT * FROM public.facturas_noanuladas WHERE periodo_tributario = 202509")

# TABLA FACTURAS DE SALES SYSTEM
facturas = pd.read_sql(query, con=warehouse, parse_dates=['fecha_emision','fecha_vencimiento'], dtype_backend="pyarrow")

# TABLA PROVEEDORES DE SALES SYSTEM
proveedores = pd.read_sql("SELECT tipo_proveedor, numero_documento, alias FROM proveedores", con=salessystem, dtype_backend="pyarrow")

# TABLA CLIENTES DE SALES SYSTEM
clientes = pd.read_sql("SELECT * FROM customers", con=salessystem, dtype_backend="pyarrow")

**FACTURAS AGRUPADAS POR ENTIDAD, TOTAL Y CANTIDAD DE COMPROBANTES**

In [24]:
# FACTURAS AGRUPADAS POR PROVEEDOR(ruc), SUMA IGV, SUMA VALOR, SUMA IGV Y CONTEO
result = facturas.groupby('ruc').agg(
    columna1_sum=('valor', 'sum'),
    columna2_sum=('igv', 'sum'),
    count=('ruc', 'size')
).reset_index()
# TOTAL = SUMA IGV + SUMA SUMA VALOR
result['total']=result['columna1_sum']+result['columna2_sum']
# ELIMINAR COLUMNAS SUMA IGV Y SUMA VALOR
result.drop(['columna1_sum', 'columna2_sum'], axis=1, inplace=True)

In [25]:
# FILTRAR SOLO PROVEEDORES TIPO 1 Y 2
proveedores_tipo1y2 = proveedores[proveedores['tipo_proveedor'] < '3']
# FILTRAR SOLO PROVEEDORES TIPO 3
proveedores_tipo3 = proveedores[proveedores['tipo_proveedor'] == '3']
# FILTRAR SOLO CLIENTES INTERNOS REGISTRADOS
clientes_internos = clientes[clientes['observaciones'].str.contains('INTERNO', case=False, na=False)]
# FILTRAR SOLO CLIENTES EXTERNOS REGISTRADOS
clientes_externos = clientes[~clientes['observaciones'].str.contains('INTERNO|PROVEEDOR', case=False, na=False)]

In [26]:
# FILTRAR FACTURAS EMITIDAS A PROVEEDORES TIPO 1 Y 2
filtro1_proveedores = facturas[facturas['numero_documento'].isin(proveedores_tipo1y2['numero_documento'].astype(str))]

In [27]:
# FILTRAR FACTURAS EMITIDAS A PROVEEDORES TIPO 3
filtro2_proveedores = facturas[facturas['numero_documento'].isin(proveedores_tipo3['numero_documento'].astype(str))]

In [28]:
# FILTRAR FACTURAS EMITIDAS A CLIENTES INTERNOS REGISTRADOS
filtro_clientesinternos = facturas[facturas['numero_documento'].isin(clientes_internos['ruc'].astype(str))]

In [29]:
# FILTRAR FACTURAS EMITIDAS A OTROS CLIENTES O A CLIENTES NO REGISTRADOS
filtro_otrosclientes = facturas[~facturas['numero_documento'].isin(clientes['ruc'].astype(str))]
filtro_otrosclientes['numero_documento'].drop_duplicates()

Series([], Name: numero_documento, dtype: string[pyarrow])

In [30]:
# FILTRAR FACTURAS EMITIDAS A CLIENTES EXTERNOS
filtro_clientesexternos = facturas[facturas['numero_documento'].isin(clientes_externos['ruc'].astype(str))]

In [31]:
print('Cantidad de facturas emitidas por facturacion: '+str(len(facturas['cui'])))
print('Cantidad de facturas emitidas a proveedores tipo 1 y 2: '+str(len(filtro1_proveedores['cui'])))
print('Cantidad de facturas emitidas a proveedores tipo 3: '+str(len(filtro2_proveedores['cui'])))
print('Cantidad de facturas emitidas a clientes internos: '+str(len(filtro_clientesinternos['cui'])))
print('Cantidad de facturas emitidas a clientes externos recurrentes: '+str(len(filtro_clientesexternos['cui'])))
print('Cantidad de facturas emitidas a nuevos clientes: '+str(len(filtro_otrosclientes['cui'])))

Cantidad de facturas emitidas por facturacion: 559
Cantidad de facturas emitidas a proveedores tipo 1 y 2: 35
Cantidad de facturas emitidas a proveedores tipo 3: 0
Cantidad de facturas emitidas a clientes internos: 34
Cantidad de facturas emitidas a clientes externos recurrentes: 525
Cantidad de facturas emitidas a nuevos clientes: 0


In [32]:
# FILTRO FACTURAS CON GUIA
filtro_facturaconguia = facturas[facturas['tipo_documento_referencia'] == 9]
# FILTRO FACTURAS SIN GUIA
filtro_facturasinguia = facturas[facturas['tipo_documento_referencia'] == 1]

In [33]:
print('Cantidad de facturas con guia asociada: '+str(len(filtro_facturaconguia['cui'])))
print('Cantidad de facturas sin guia: '+str(len(filtro_facturasinguia['cui'])))

Cantidad de facturas con guia asociada: 552
Cantidad de facturas sin guia: 7


In [34]:
# TABLA PEDIDOS ORDENADO POR ADQUIRIENTE/CLIENTE FILTRADO POR PERIODO '202406'
pedidos = pd.read_sql("SELECT * FROM pedidos WHERE periodo = 202509",salessystem, parse_dates=['fecha_pedido'], dtype_backend="pyarrow").sort_values(by='adquiriente', ascending=True)
# PEDIDOS FILTRADOS POR PEDIDOS ENTREGADOS Y/O EMITIDOS
pedidos_preparados = pedidos[pedidos['estado'].str.contains('ENTREGADO|EMITIDO', case=False, na=False)]

In [35]:
# FACTURAS FILTRADAS POR PEDIDOS REGISTRADOS COMO EMITIDO/ENTREGADO
facturas_pedidos_preparados=facturas[facturas['numero_documento'].isin(pedidos_preparados['adquiriente'].unique().astype(str))]
# FACTURAS FILTADAS POR PEDIDOS REGISTRADOS AGRUPADAS POR ADQUIRIENTE(numero_documento)
pedidos_verificados = facturas_pedidos_preparados.groupby('numero_documento').agg(
    columna1_sum=('valor', 'sum'),
    columna2_sum=('igv', 'sum'),
    count=('ruc', 'size')
).reset_index()
pedidos_verificados['total']=pedidos_verificados['columna1_sum']+pedidos_verificados['columna2_sum']
pedidos_verificados.drop(['columna1_sum', 'columna2_sum'], axis=1, inplace=True)

In [36]:
pedidos_verificados

,numero_documento,count,total


In [37]:
# POR CADA ADQUIRIENTE DEL DATAFRAME pedidos_preparados
for adquiriente in pedidos_preparados['adquiriente'].unique():
    # FILTRAR PEDIDOS POR ADQUIRIENTE
    pedido = pedidos_preparados[pedidos_preparados['adquiriente'] == adquiriente]
    # FACTURAS EMITIDAS PARA LOS PEDIDOS PREPARADOS FILTRADAS POR ADQUIRIENTE (numero_documento)
    total_facturas_adquiriente = pedidos_verificados[pedidos_verificados['numero_documento'] == str(adquiriente)]
    
    # SUMAR TOTAL POR ADQUIRIENTE
    total_por_adquiriente = total_facturas_adquiriente['total'].sum()
    
    # SI EL ADQUIRIENTE TIENE MAS DE 1 PEDIDO EN EL PERIODO
    if len(pedido) > 1:
        print('Varios pedidos de un adquiriente')
    # SI SOLO TIENE 1 PEDIDO    
    else: 
        # OBTENER EL CODIGO DE PEDIDO
        pedido1 = pedido['cod_pedido'].values[0]
        # OBTENER EL IMPORTE TOTAL DEL PEDIDO
        importe = pedido['importe_total'].values[0]
        
        print(f"Pedido codigo {pedido1} por {importe} emitido por {total_por_adquiriente}")

In [38]:
# PEDIDOS EMITIDOS POR FACTURACION AGRUPADOS Y ORDENADOS POR PROVEEDOR(RUC) FILTRADOS POR PERIODO
emitidos_facturacion = pd.read_sql("SELECT ruc,count(distinct concat(cod_pedido,cuo)), SUM(round(cantidad * precio_unit *1.18,2)) AS total FROM facturas WHERE CAST(DATE_FORMAT(emision, '%%Y%%m') AS UNSIGNED) = 202509 GROUP BY ruc ORDER BY ruc", con=salessystem)

In [39]:
emitidos_facturacion

,ruc,"count(distinct concat(cod_pedido,cuo))",total
0,10065539964,3,5000.08
1,10067597937,2,2400.04
2,10070320814,7,6000.05
3,10085917213,8,10000.57
4,10097937724,14,58023.73
5,10105880818,10,8100.23
6,10258333450,4,5002.61
7,10297073058,7,3007.35
8,10415173101,32,55000.85
9,10470078923,17,19005.44


In [40]:
#FACTURAS QUE NO SON A PROVEEDORES
facturas_cliente=facturas[~facturas['numero_documento'].isin(proveedores['numero_documento'].astype(str))]

In [41]:
facturas_cliente_conguia=facturas_cliente[
    (facturas_cliente['tipo_documento_referencia'] == 9) &
    (~facturas_cliente['numero_documento_referencia'].str.startswith('0001'))
]
facturas_cliente_conguia.loc[:, 'numero_documento_referencia'] = facturas_cliente_conguia['numero_documento_referencia'].apply(lambda x: int(x[6:-1]))

In [42]:
facturas_cliente_conguia.sort_values(by=['ruc','numero_documento_referencia'], ascending=True)

,id,ruc,subdiario,periodo_tributario,tipo_operacion,tipo_comprobante,fecha_emision,fecha_vencimiento,numero_serie,numero_correlativo,...,tipo_comprobante_modificado,numero_serie_modificado,numero_correlativo_modificado,glosa,cui,observaciones,cuenta_contable,igv,tipo_documento_referencia,numero_documento_referencia
274,13768,10406916087,5,202509,1,1,2025-09-20,NaT,E001,205,...,<NA>,<NA>,<NA>,<NA>,26c4ceff701E001205,PARSER,<NA>,201.6,9,49
277,13771,10406916087,5,202509,1,1,2025-09-22,NaT,E001,206,...,<NA>,<NA>,<NA>,<NA>,26c4ceff701E001206,PARSER,<NA>,116.68,9,50
392,13886,20606283858,5,202509,1,1,2025-09-05,NaT,E001,263,...,<NA>,<NA>,<NA>,<NA>,4cc3af05201E001263,PARSER,<NA>,2024.64,9,254
395,13889,20606283858,5,202509,1,1,2025-09-09,NaT,E001,264,...,<NA>,<NA>,<NA>,<NA>,4cc3af05201E001264,PARSER,<NA>,247.57,9,255
398,13892,20606283858,5,202509,1,1,2025-09-10,NaT,E001,265,...,<NA>,<NA>,<NA>,<NA>,4cc3af05201E001265,PARSER,<NA>,278.39,9,256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,13897,20614580951,5,202509,1,1,2025-09-30,NaT,E001,266,...,<NA>,<NA>,<NA>,<NA>,4ccb98ad701E001266,PARSER,<NA>,5882.4,9,262
406,13900,20614580951,5,202509,1,1,2025-09-30,NaT,E001,267,...,<NA>,<NA>,<NA>,<NA>,4ccb98ad701E001267,PARSER,<NA>,2684.74,9,263
409,13903,20614580951,5,202509,1,1,2025-09-30,NaT,E001,268,...,<NA>,<NA>,<NA>,<NA>,4ccb98ad701E001268,PARSER,<NA>,127.86,9,264
412,13906,20614580951,5,202509,1,1,2025-09-30,NaT,E001,269,...,<NA>,<NA>,<NA>,<NA>,4ccb98ad701E001269,PARSER,<NA>,6298.34,9,265
